This Code calculates the failure criterion for Phobos (Drucker-Prager) and compares it to various estimates of strength of Phobos. The code can be used to generate Figure 2a and Figure 2b in the paper 

In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy import constants
from scipy import integrate

G_val = constants.G.si.value

M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3        # Mars, m
rho_mars = 3933            # kg/m^3, Mars density
V_phobos = 5689*1e9        # +/- 60*1e9 m^3, Phobos Volume
M_phobos = 1.065*(10**16.) # Phobos Mass, +/- .015 kg
R_phobos = 11.3*1e3        # Phobos Radius, m
rho_phobos = 1860.         # Phobos density, +/- 31 kg/m^3
ecc_phobos =  0.01511      # Current eccentricity of Phobos
a_phobos = 9378*1e3        # Semi-major Axis of Phobos, m 
w_phobos = 1./27562.       # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

k2_mars = .148            # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars = 88.              # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_monolith = 8e4  # Dimensionless, from 8e4 - 8e6
mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100.                         # Can be between 50 - 100

# Some terms for the tidal evolution equation (See text for details)

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e,w_s,E] - for moon/Phobos , E is the Energy, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.):
        dt_e =0.
    if (X[0] <=1.):
        dt_a =0.
        dt_e =0.
    return array([dt_a,dt_e])

max_t =80.*1e6*3.154e7                            # unit is per seconds
dt = 1e4*3.154e7
X_init = array([a_phobos/R_mars,ecc_phobos])               # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
t_vX.append(0.)
while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	t_vX.append(abc.t)
	#print abc.t/max_t
    
a_int = array(a_intX)
e_int = array(e_intX)
t_v = array(t_vX)

a_int_orig = a_int.copy()
e_int_orig = e_int.copy()


In [3]:
# Specify the Shape properties of Phobos and calcuate the terms for Drucker-Prager failure criterion ..

Shape_phobos_a = 13.03*1e3   
Shape_phobos_b = 11.40*1e3 
Shape_phobos_c = 9.14*1e3 
Shape_phobos_alpha = Shape_phobos_c/Shape_phobos_a
Shape_phobos_beta = Shape_phobos_b/Shape_phobos_a

Ax_phobos = Shape_phobos_alpha*Shape_phobos_beta*.878915
Ay_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.03733
Az_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.34263
w_centr = sqrt(G_val*M_mars/(a_int*R_mars)**3.)

Term1 = rho_phobos*w_centr*w_centr + (8.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ax_phobos*rho_phobos**2.
Term2 = rho_phobos*w_centr*w_centr - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3.-2.*pi*G_val*Ay_phobos*rho_phobos**2.
Term3 =  -1.*(4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Az_phobos*rho_phobos**2.
sigma_x = Term1*Shape_phobos_a**2./5.
sigma_y = Term2*Shape_phobos_b**2./5.
sigma_z = Term3*Shape_phobos_c**2./5.

J2_term = sqrt((sigma_x-sigma_y)**2. + (sigma_y-sigma_z)**2. + (sigma_z-sigma_x)**2. )/sqrt(6.)

r_mean_particle_size = 1e-6              # 1 micron, Typical size of regolith particles 
Y_minimum = 1.56e-4/r_mean_particle_size 
# ~ 100 Pascal - Expected strength of Phobos just due to regolith particles using model from Sanchez and Scheeres 2014


In [4]:
# plotting 

import numpy as np
import pylab as plt

fig_width_pt = 2.*130.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean      # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 5,
          'xtick.labelsize': 6.5,
          'axes.linewidth': .3,
          'ytick.labelsize': 6.5}
pylab.rcParams.update(params)
import matplotlib.gridspec as gridspec

fig,axes = plt.subplots(figsize=(fig_width,fig_height),dpi=250)
c=['r','b','k','g','c','y']


# Use the Inscribed version of Drucker Prager failure criterion

# Calculate the required strength of Phobos for friction angle = 45 degrees
friction_ang = (45.)*pi/180.
s = 3.*sin(friction_ang)/sqrt(9. +3.*sin(friction_ang)**2.)
I_term = s*(sigma_x + sigma_y + sigma_z)/3.
Cohesion_needed_term = J2_term + I_term
k_term = 3.*cos(friction_ang)/sqrt(9. +3.*sin(friction_ang)**2.)
term1 = Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term # in Mega Pascal

term1[term1 < 0] = min(term1[term1 > 0])
axes.semilogy(a_int[a_int>1.+160./R_mars],term1,'-',color='g',linewidth = .7)

# Calculate the required strength of Phobos for friction angle = 25 degrees
friction_ang = (25.)*pi/180.
s = 3.*sin(friction_ang)/sqrt(9. +3.*sin(friction_ang)**2.)
I_term = s*(sigma_x + sigma_y + sigma_z)/3.
Cohesion_needed_term = J2_term + I_term
k_term = 3.*cos(friction_ang)/sqrt(9. +3.*sin(friction_ang)**2.)
term2 = Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term
term2[term2 < 0] = min(term2[term2 > 0])

x_value = a_int[a_int>1.+160./R_mars]
axes.semilogy(a_int[a_int>1.+160./R_mars],term2,'-',color='g',linewidth = .7)
axes.fill_between(x_value,term1,term2,facecolor='green', alpha=0.25)

axes.set_xlim([1.0674454227893777,2.7667797610267])
axes.set_ylabel('Critical Yield\n Strength(MPa)',fontsize=10,color='blue')
axes.set_xlabel('a/R$_{Mars}$',fontsize=10,labelpad=3.5)
plt.show()

################ Estimates of Strength of Phobos
## Hoek-Brown Model - estimate for strength of Phobos. 
# The estimates are calculated for a range of GSI, mi values (See text for details)

D = 1.  
mi = 5. 
GSI = 5. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
Y_0 = 50.0*1e6 # Pascal (10-100 Mpa)

Y_strength_model = zeros([4,1])
for i in range(0,4):  # Use a range of friction angles ..
    friction_ang = (25. + i*7.)*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_strength_model[i] = (Y_minimum+ Y_damaged)/1e6
    #print(friction_ang/(pi/180.),(Y_minimum+ Y_damaged)/1e6)
    
Y_strength_model_Hoek1 = Y_strength_model.copy()

D = 1. 
mi = 15. 
GSI = 25.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

Y_0 = 50.0*1e6  # Pascal (10-100 Mpa)

Y_strength_model = zeros([4,1])
for i in range(0,4):
    friction_ang = (25. + i*7.)*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    
    # Stress models :
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    Y_strength_model[i] = (Y_minimum+ Y_damaged)/1e6

from matplotlib.patches import Ellipse, Polygon
p2 = axes.add_patch(Polygon([[1.0674454227893777,Y_strength_model[0]],[1.0674454227893777,Y_strength_model_Hoek1[3]],
                             [2.7667797610267,Y_strength_model_Hoek1[3]],[2.7667797610267,Y_strength_model[0]]], closed=True,
                      fill=False, hatch='/',linewidth=.5,color='black'))


plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.15)


# Model 2 : Holsapple 2008, using rotation speeds of asteroids to estimate minimum required cohesion

k_term_holsapple_model = (0.025*rho_phobos**2./sqrt(R_phobos))
k_term_out = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))

axes.semilogy(a_int[a_int>1.+160./R_mars],a_int[a_int>1.+160./R_mars]*0. + (Y_minimum)/1e6 + k_term_holsapple_model/k_term_out/1e6,'--',color='k',linewidth=1.15)

# Model 3 : Tagish lake meteorite. Estimate of strength based on ram pressures at location of atmospheric breakup
for i in range(0,4):
    friction_ang = (25. + i*7.)*pi/180.
    strength_M3 = 0.25*1e6*(1. - sin(friction_ang))/2./cos(friction_ang)
    axes.semilogy(a_int[a_int>1.+160./R_mars],a_int[a_int>1.+160./R_mars]*0. + strength_M3/1e6,'--',color='r',linewidth=1.15)


# Model 4 : Strength required to support Topography
for i in range(0,4):
    friction_ang = (25. + i*7.)*pi/180.
    strength_M4a = 23000.*(1. - sin(friction_ang))/2./cos(friction_ang)
    axes.semilogy(a_int[a_int>1.+160./R_mars],a_int[a_int>1.+160./R_mars]*0. + strength_M4a/1e6,'--',color='b',linewidth=1.15)
    #print('r',strenth_tagish_lake/1e6)

for i in range(0,4):
    friction_ang = (25. + i*7.)*pi/180.
    strength_M4b = 47000.*(1. - sin(friction_ang))/2./cos(friction_ang)
    axes.semilogy(a_int[a_int>1.+160./R_mars],a_int[a_int>1.+160./R_mars]*0. + strength_M4b/1e6,'--',color='b',linewidth=1.15)

p1 = axes.add_patch(Polygon([[-1,.1],[-1.3,.2],[-1.1,.3],[-2,.3]], closed=True,
                      fill=True,linewidth=.5,color='green',alpha=.25))

legend([p1],[r'$\theta_{f}=\,25^{\circ} -\, 45^{\circ} $',],loc='best', framealpha=.75)

annotate('$Y_{strength}$ - Hoek-Brown model', [1.7,.997],
                    xycoords='data',xytext=(15.,-10.), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
                                        connectionstyle="arc3,rad=0.15",color='k',alpha=0))

annotate('Minimum $Y_{strength}$ model', [1.57,k_term_holsapple_model/k_term_out/1e6],
                    xycoords='data',xytext=(-50.,-6), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=1,
                                        connectionstyle="arc3,rad=-0.01",color='k',alpha=0))

annotate('$Y_{strength}$ - Tagish Lake', [1.9,.042],
                    xycoords='data',xytext=(15.,-10.), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
                                        connectionstyle="arc3,rad=0.15",color='k',alpha=0))

annotate('$Y_{strength}$ - Topogaphy', [1.2,.0042],
                    xycoords='data',xytext=(15.,-10.), textcoords='offset points',
                    fontsize=6,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=5,
                                        connectionstyle="arc3,rad=0.15",color='k',alpha=0))


ylim([3e-4,10])

axR = axes.twinx()
axR.semilogy(a_int[a_int>1.+160./R_mars],term1,'-',color='b',linewidth = 0)
axR.set_ylabel('Model Yield\n Strength(MPa)',fontsize=10,color='k')
axR.set_xlim([1.0674454227893777,2.7667797610267])
axR.set_ylim([3e-4,10])

axes.tick_params(axis='y', colors='blue')
xlim([1.0,2.6])
plot([1.067,1.067],[3e-4,10],'k-')

/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontweight is deprecated and replaced with font.weight; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
savefig("Fig2a_New_Version1.pdf",dpi=1200,bbox_inches='tight')


In [6]:
from scipy import interpolate
from scipy.interpolate import interp1d


M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3    # Mars, m
rho_mars = 3933        #kg/m^3
V_phobos = 5689*1e9    # +/- 60*1e9 m^3
M_phobos = 1.065*(10**16.) # Phobos, +/- .015 kg
R_phobos = 11.3*1e3  # Phobos, m
rho_phobos = 1860.   # +/- 31 kg/m^3
ecc_phobos =  0.01511
a_phobos = 9378.*1e3 # Semi-major Axis, m 
w_phobos = 1./27562. # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

Shape_phobos_a = 13.03*1e3  # m
Shape_phobos_b = 11.40*1e3  # m
Shape_phobos_c = 9.14*1e3   # m
Shape_phobos_alpha = Shape_phobos_c/Shape_phobos_a
Shape_phobos_beta = Shape_phobos_b/Shape_phobos_a

Ax_phobos = Shape_phobos_alpha*Shape_phobos_beta*.878915
Ay_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.03733
Az_phobos = Shape_phobos_alpha*Shape_phobos_beta*1.34263
w_centr = sqrt(G_val*M_mars/(a_int*R_mars)**3.)

Term1 = rho_phobos*w_centr*w_centr + (8.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Ax_phobos*rho_phobos**2.
Term2 = rho_phobos*w_centr*w_centr - (4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3.-2.*pi*G_val*Ay_phobos*rho_phobos**2.
Term3 =  -1.*(4.*pi/3.)*G_val*rho_mars*rho_phobos/(a_int)**3. -2.*pi*G_val*Az_phobos*rho_phobos**2.
sigma_x = Term1*Shape_phobos_a**2./5.
sigma_y = Term2*Shape_phobos_b**2./5.
sigma_z = Term3*Shape_phobos_c**2./5.

J2_term = sqrt((sigma_x-sigma_y)**2. + (sigma_y-sigma_z)**2. + (sigma_z-sigma_x)**2. )/sqrt(6.)

r_mean_particle_size = 1e-6 # 1 micron 
Y_minimum = 1.56e-4/r_mean_particle_size # ~ 100 Pascal

import numpy as np
import pylab as plt

fig_width_pt = 2.*130.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean      # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 5,
          'xtick.labelsize': 7.5,
          'axes.linewidth': .3,
          'ytick.labelsize': 7.5}
pylab.rcParams.update(params)
#fig_size =  [fig_width,fig_height]
import matplotlib.gridspec as gridspec

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True,figsize=(fig_width,fig_height),dpi=250)
clf()

fric_ang_all = linspace(5,80,1000)
all_fric_angl = zeros(1000)
loc_fric_angl = zeros(1000)

# Hoek-Brown model for strength of Phobos 
# The code calucates the intersection between the Drucker-Prager failure criterion 
# and the estimates of strength of Phobos using the Hoek-Brown model as a function of friction angle

# First consider the strength of Phobos in the weak, damaged region (See Figure 2a for details)

D = 1. 
mi = 5. 
GSI = 5.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
Y_0 = 50*1e6 # Pascal (10-100 Mpa)

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

loc_fric_angl_S1  = loc_fric_angl.copy()
all_fric_angl_S1 = all_fric_angl.copy()

# Just smooth the intersection boundaries by interpolation ...

tck_1 = interpolate.splrep(all_fric_angl_S1,loc_fric_angl_S1, s=.05)
all_fric_angl_new1 = np.linspace(5., 80, 90)
ynew_1 = interpolate.splev(all_fric_angl_new1, tck_1, der=0)

##############################
##############################

D = 1. 
mi = 5.
GSI = 12.
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]


# Just smooth the intersection boundaries by interpolation ...
f2 = interp1d(all_fric_angl,loc_fric_angl)
all_fric_angl_new = np.linspace(5., 25., 90)

tck = interpolate.splrep(all_fric_angl,loc_fric_angl, s=.05)
all_fric_angl_new2 = np.linspace(25., 80, 90)
ynew = interpolate.splev(all_fric_angl_new2, tck, der=0)

loc_fric_angl_S2  =  hstack([f2(all_fric_angl_new),ynew])
all_fric_angl_S2 = hstack([all_fric_angl_new,all_fric_angl_new2])


##############################
##############################
# Plot the region ...

plot(ynew_1, all_fric_angl_new1, loc_fric_angl_S2, all_fric_angl_S2,'k-',linewidth=1.1,color='black' )
ax=gca()

f3 = interp1d(all_fric_angl_S2,loc_fric_angl_S2)
loc_fric_angl_S3  =  f3(all_fric_angl_new1)
ax.fill_betweenx(all_fric_angl_new1, ynew_1, loc_fric_angl_S3, where=ynew_1>=loc_fric_angl_S3, hatch='/',facecolor='white',alpha=.9)


title('Hoek Model - Disruption Location',fontsize=8)
ylabel(r'$\theta_{friction} \, (deg)$',fontsize=10,labelpad=3.5)
xlabel('a$_{disruption}$/R$_{Mars}$ ',fontsize=10)

##############################
##############################


plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.2)
ylim([10,80])
xlim([1.0,2.6])

from matplotlib.patches import Rectangle
ax = gca()
ax.add_patch(Rectangle(( 1.,25.), 1.6,20.,alpha=.7,facecolor='grey',linewidth=.5))
plt.show()

######################################
######################################
######################################
# Now consider the strength of Phobos in the stronger region (See Figure 2a for details)

ynew_1 = loc_fric_angl_S2.copy()
all_fric_angl_new1 = all_fric_angl_S2.copy()

D = 1. 
mi = 5. 
GSI = 20. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

# Just smooth the intersection boundaries by interpolation ...

f2 = interp1d(all_fric_angl,loc_fric_angl)
all_fric_angl_new = np.linspace(5., 45., 90)

tck = interpolate.splrep(all_fric_angl[all_fric_angl > 45.],loc_fric_angl[all_fric_angl > 45.], s=.1)
all_fric_angl_new2 = np.linspace(45., 80, 90)
ynew = interpolate.splev(all_fric_angl_new2, tck, der=0)

loc_fric_angl_S2  =  hstack([f2(all_fric_angl_new),ynew])
all_fric_angl_S2 = hstack([all_fric_angl_new,all_fric_angl_new2])

plot(ynew_1, all_fric_angl_new1, loc_fric_angl_S2, all_fric_angl_S2,'k-',linewidth=1.1,color='black' )

######################################
ynew_1 = loc_fric_angl_S2.copy()
all_fric_angl_new1 = all_fric_angl_S2.copy()

D = 1. 
mi = 5. 
GSI = 25. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

# Just smooth the intersection boundaries by interpolation ...

f2 = interp1d(all_fric_angl,loc_fric_angl)
all_fric_angl_new = np.linspace(5., 51., 90)

tck = interpolate.splrep(all_fric_angl[all_fric_angl > 50.],loc_fric_angl[all_fric_angl > 50.], s=.1)
all_fric_angl_new2 = np.linspace(51., 80, 90)
ynew = interpolate.splev(all_fric_angl_new2, tck, der=0)

loc_fric_angl_S2  =  hstack([f2(all_fric_angl_new),ynew])
all_fric_angl_S2 = hstack([all_fric_angl_new,all_fric_angl_new2])

plot(ynew_1, all_fric_angl_new1, loc_fric_angl_S2, all_fric_angl_S2,'k-',linewidth=1.1,color='black' )
ax=gca()

f3 = interp1d(all_fric_angl_S2,loc_fric_angl_S2)
loc_fric_angl_S3  =  f3(all_fric_angl_new1)
ax.fill_betweenx(all_fric_angl_new1, ynew_1, loc_fric_angl_S3, where=ynew_1>=loc_fric_angl_S3, hatch='/',facecolor='white',alpha=.9)


######################################
ynew_1 = loc_fric_angl_S2.copy()
all_fric_angl_new1 = all_fric_angl_S2.copy()


D = 1. 
mi = 15. 
GSI = 25. 
mb = mi*exp((GSI - 100.)/(28.-14.*D))
s_hoek = exp((GSI - 100.)/(9.-3.*D))
a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))

for i in range(0,1000):
    friction_ang = fric_ang_all[i]*pi/180.
    s = 2.*sin(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    I_term = s*(sigma_x + sigma_y + sigma_z)
    Cohesion_needed_term = J2_term + I_term
    k_term = 6.*cos(friction_ang)/sqrt(3.)/(3. - sin(friction_ang))
    Term_hoek = ((mb + 4.*s_hoek -a_hoek*(mb- 8.*s_hoek))*(mb/4.  +s_hoek)**(a_hoek-1.))/(2.*(a_hoek+1.)*(a_hoek+2.))
    Y_damaged = Y_0*Term_hoek*(1. - sin(friction_ang))/2./cos(friction_ang)
    tmp1 = argmin(abs(Cohesion_needed_term[a_int>1.+160./R_mars]/1e6/k_term - Y_damaged/1e6))
    all_fric_angl[i] = friction_ang/(pi/180.)
    loc_fric_angl[i] = a_int[tmp1]

# Just smooth the intersection boundaries by interpolation ...

f2 = interp1d(all_fric_angl,loc_fric_angl)
all_fric_angl_new = np.linspace(5., 61., 90)

tck = interpolate.splrep(all_fric_angl[all_fric_angl > 60.],loc_fric_angl[all_fric_angl > 60.], s=.1)
all_fric_angl_new2 = np.linspace(61., 80, 90)
ynew = interpolate.splev(all_fric_angl_new2, tck, der=0)

loc_fric_angl_S2  =  hstack([f2(all_fric_angl_new),ynew])
all_fric_angl_S2 = hstack([all_fric_angl_new,all_fric_angl_new2])

plot(ynew_1, all_fric_angl_new1, loc_fric_angl_S2, all_fric_angl_S2,'k-',linewidth=1.1,color='black' )
ax=gca()

f3 = interp1d(all_fric_angl_S2,loc_fric_angl_S2)
loc_fric_angl_S3  =  f3(all_fric_angl_new1)
ax.fill_betweenx(all_fric_angl_new1, ynew_1, loc_fric_angl_S3, where=ynew_1>=loc_fric_angl_S3, hatch='/',facecolor='white',alpha=.9)


/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontweight is deprecated and replaced with font.weight; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
annotate('Likely range \n      for'+r' $\theta_{f}$', [2.14,36.1,],
                    xycoords='data',xytext=(-20,27.), textcoords='offset points',
                    fontsize=9,
                    arrowprops=dict(arrowstyle='->',fc="0.9",lw=.6,shrinkA=0, shrinkB=.6,
                                        connectionstyle="arc3,rad=0.1",color='k',alpha=1.))


savefig("Fig2b_New__Version1.pdf",dpi=900)

#py.plot(arr[:,0], arr[:,1], 'o', alpha=0.1, rasterized=True)
#py.savefig('dots.pdf', dpi=400)


In [7]:
# This code snippet calcuates the expected failure location of Phobos 
# using the Hoek-Brown model both for strength of Phobos as well as the failure criterion 

## Hoek Model ... 
D = 1. 
sigma_ci = 50.0*1e6 # Pascal (100 Mpa)
inters_hoek = zeros([10,10])
for i in range(0,10):
    mi = 5. +i*(11.)/10. 
    for k in range(0,10):
        GSI = 5. +k*(14.)/10.
        
        mb = mi*exp((GSI - 100.)/(28.-14.*D))
        s_hoek = exp((GSI - 100.)/(9.-3.*D))
        a_hoek = .5 + (1./6.)*(exp(-GSI/15.) - exp(-20./3.))
        
        sigma_3 = minimum(minimum(-sigma_x,-sigma_y),-sigma_z) # gives sigma_3
        sigma_1 = maximum(maximum(-sigma_x,-sigma_y),-sigma_z) # gives sigma_1
        
        # Basic Hoek-Brown criterion 2002
        term_hoek_1 = (mb*sigma_3/sigma_ci + s_hoek)
        #print(min(term_hoek_1),max(term_hoek_1),s_hoek)
        
        term_lhs = sigma_3 + sigma_ci*(term_hoek_1)**a_hoek
        term_rhs = sigma_1
        term_lhs[isnan(term_lhs)] = 0.
        msc1 = argmin(abs(term_lhs - term_rhs))
        msc2 = min(abs(term_rhs - term_lhs))
        inters_hoek[i,k] = a_int[msc1]

# the minimum and maximum value for semi-major axis where Phobos is expected to tidally break-up
print(amin(inters_hoek),amax(inters_hoek)) # in R_mars units

1.97908531244 2.41608820912


/home/tmittal2/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py:24: RuntimeWarning: invalid value encountered in power
